# 画像処理とディープラーニング

画像分類などに用いられるディープラーニングである畳み込みニューラルネットワーク、(Convolutional Neural Network ; 以下 CNN)　の概要とその構造を学びます。  画像デー タをどのように数値に落とし込むのか、また CNN と従来の機械学習の違いについても学び、CNN の理解を深めます。

## 本章の構成

- 画像処理の基礎
- Convolutional Neural Network (CNN) の概要と構造

## 画像処理の基礎

CNN は画像分類や物体検出などの画像に関するディープラーニングのモデルになります。画像データがどういったものなのか、そして CNN とはどういったモデルなのかについて学びます。

### 画像の構成要素

![画像の構成要素](http://drive.google.com/uc?export=view&id=1A4V0tsbY84GsDW-NGbGChlhotYFGz8j1)

画像は高さ (Height) と幅 (width)、そして奥行き (channel) という情報を持っています。一般的に用いられるのは、**RGB** という光の三原色である赤 (Red)、緑 (Green)、青 (Blue) の 3 枚の画像を重ね合わせ色を表現するタイプの画像になります。  

この画像のひとつの正方形を **ピクセル (pixel)** と表現し、各ピクセルにはそれぞれ**輝度**と呼ばれる光の明るさを表した値が格納されています。ピクセルがどれくらい集まっているのかを表す単位は**画素**と呼ばれ、縦と横に並ぶピクセルの数で表現されます。また、輝度はコンピュータで扱う場合には $0$ ~ $255$ の符号なし整数 (unsigned integer) の 8bit で表現することが一般的です。  

高さ (Height)、幅 (Width)、奥行き (Channel) の 3 つが画像には存在することがわかりました。こちらは数学的には行列をまとめたものであるため、**テンソル** **(Tensor)** に相当します。  

### 画像の特徴抽出

画像がどのような構成要素でできているのかを把握することができました。さて、ここで考えたいことは、これまでは全結合のニューラルネットワークを学びましたが、**画像をどのように入力値として全結合のニューラルネットワークに取り入れるか**です。  

画像はテンソルの形のデータになります。しかし、全結合のニューラルネットワークに取り入れるためには、ベクトルである必要があります。行列でもなく、テンソルの形である画像をどのようにベクトルに変換できるか考えてみましょう。  

CNN がどのように画像を取扱うのかについて学ぶ前に、これまでどのように画像をベクトルに変換を行っていたのか確認します。  


#### 画像を切り取る

1 つ目の方法は単純に画像を横方向に切り取り、並べると行った方法になります。  
下記の画像を確認して下さい。  

![画像を切り取る](http://drive.google.com/uc?export=view&id=1qkYQiVFo5B2iqYw8l3IDYlxVWcSw1f3T)

このように画像を横方向（もしくは縦方向）に切り取り 1 並びにすることによってテンソルの画像をベクトルに変換することが可能です。しかし、この方法には 2 つの欠点があります。  

- 画像内の物体の位置を考慮していない : 単純に切り取っているだけのため、画像内の位置は考慮されていません。
- パラメータの数が膨大になる : ニューラルネットワークの入力として取り扱った場合、パラメータの数が膨大になることが考えれます。

2 点目の欠点について確認します。例えば、$100 × 100$ の高さ、幅を持つカラー画像を想定します。その場合の画像全体の画素数は、

$$
\begin{align}
100 × 100 × 3 = 30000
\end{align}
$$

となります。これを入力値として、次の層のノード数を1000 とした場合、パラメータの数はバイアスも考慮すると、

$$
\begin{align}
30000 × 1000 + 1000 = 30,001,000
\end{align}
$$

となり、わずか 2 層の全結合ニューラルネットワークで約 3 千万個のパラメータを調整しなければいけません。

#### ヒストグラムを取る

次の手段として**ヒストグラム**を採用することを考えてみましょう。ヒストグラムは画像の輝度の値をカウントします。下の図のように、ヒストグラムではどのような画像サイズに対しても、$0$ ~ $255$ の $256$ 次元のベクトル（RGB の 3 チャネルを考慮した場合、$256×3=768$）となるため、前述のパラメータの数を大幅に減らすことができます。  

このように、オリジナルの入力の値を別の値に変換して取り扱いやすくする処理を**特徴抽出**と呼び、取り出された値を**特徴量**といいます。  

![ヒストグラム](http://drive.google.com/uc?export=view&id=1lsdJDjJYsb4IP8wtckZCgfPrByRUenT7)

ヒストグラムを用いての特徴量抽出には下記の特徴があります。  

- 調整するパラメータ量が比較的少ない（単純に切り取る方法と比較して）
- 画像内の物体の位置の移動に強い（平行移動に強い）
- 画像内の物体の回転に強い

色の輝度のカウントを用いるため、画像内で同じ**平行移動**や**回転**に対してもヒストグラムは変化しないため上記のような特徴を挙げる事ができます。しかし、このヒストグラムを用いる方法でもまだ欠点は存在します。  

- 画像内の物体の位置を考慮していない

ヒストグラムは輝度をカウントする方法です。例えば人の顔写真を数値として取り扱う場合、肌色が何個あるのか、目の黒色が何個あるのかといった数は特徴として取り扱えますが、目がどこにあるのかといった情報は失われてしまいます。  

#### CNN 以前の画像分類のモデル

実際に特徴抽出を利用した画像処理の流れを解説します。下記の図では例題として、画像を人間か犬か猫の 3 種類に分類したい問題設定を想定します。これまでどのように画像をベクトルに変換し、分類を行っていたのか確認します。下記は手順の一例を示したものになります。    
 
![古典的画像分類モデル](http://drive.google.com/uc?export=view&id=1lBdt-sQkdEk-H7fHQPDfHQQNIY1vI-eW)

-  **グレースケール**変換の適用（白黒画像に変換を行う処理）
- 画像分類に必要のない背景の除去
- ヒストグラムを用いてベクトル化
- サポートベクトルマシン (SVM) やニューラルネットワークなどの分類器で分類

上記の流れがディープラーニングが登場するまでの一般的な画像分類の流れでした。こちらの方法は今でも使用されています。 
ここでこちらの方法の問題点について考えてみましょう。下記のような点を考慮する必要があります。  

- 背景除去の方法はどうするのか？
- どの分類問題設定でもグレースケール変換と背景除去の前処理だけで十分なのか？
- ヒストグラムを用いての特徴量抽出は正しいのか？

この問題点はこれまで画像処理に精通したエンジニアの経験と勘によって対処されてきました。この時点でも属人化の問題や再現性の問題が生じます。更に問題点としてはより複雑な問題設定、例えば A さんと B さんを識別するといった場合には実装が困難もしくは時間を要するといった事が挙げられます。  

これまで特徴量抽出の方法、そして古典的な画像分類モデルの問題点について確認しました。この内容を踏まえて CNN がどのようなモデルなのかについて学んでいきます。CNN を理解する上で重要な概念であるフィルタについて学びます。  

### フィルタとは

![フィルタとは](http://drive.google.com/uc?export=view&id=11J8SxYhEXSblZGXFEawOrBNXObGeD15k)

エッジ検出の処理を例に、フィルタの基礎的な概念について理解します。**フィルタ**を画像に適用すると、画像にある変換が施されます。フィルタは**カーネル**と呼ぶこともあります。  

具体的なフィルタの処理を確認します。図内のフィルタの計算方法を確認します。フィルタが適用されているのは 1 チャンネルの画像で、適当に割り振った輝度がピクセルごとに振られています。  

$$
\begin{align}
&(-1×1 + 0×2 + 1 ×3) \\\\
+&(-1×6 + 0×7 + 1 ×8) \\\\
+&(-1×11 + 0×12 + 1 ×13) \\\\
&=6
\end{align}
$$

それぞれの値を掛け合わし、そして足し合わしている事がわかります。このフィルタを適用する処理のことを**畳み込み (Convolution)** と呼びます。今回はフィルタ（カーネル）のサイズを `ksize=3`　としています（プログラミングの際にこのような変数名で登場することが多いです）。使用したフィルタを適用してなぜエッジ検出を行う事が可能なのかについては後ほど説明します。その前に畳み込みの処理方法についてもう少し確認します。    


#### ストライド

フィルタは位置をずらして、画像全体に畳み込みを適用していきます。このフィルタをずらす幅を**ストライド**と呼び、次の図では 1 つずつずらしているため、`stride=1` となります。  

![ストライド](http://drive.google.com/uc?export=view&id=1weC26ZUtP2n_RO8R3i0wTLkzbSbAZvv_)


#### パディング

また、もうひとつ考慮すべき点として、畳み込み後の画像サイズです。例えば今回は `ksize=3` で `stride=1` 、元の画像サイズが $5×5$ です。この場合の畳み込み後の画像サイズは、 $3×3$ になります。多少小さくなること自体は問題ないのですが、畳み込み後に画像サイズが変わるとプログラミングとして扱いが煩雑になってしまいます。  

そこで、画像の周囲を $0$ で埋める**パディング (padding)** と呼ばれる前処理を行います。これにより、$5×5$ の画像にパディングを適用すると $7×7$ となり、この画像に対して、フィルタサイズが 3 (`ksize=3`) の畳み込みを適用すると、畳み込み後に画像が $5×5$ となり、元の画像サイズが保たれます。

![パディング](http://drive.google.com/uc?export=view&id=1yq3isyF0VqNA8X0yY9XZf1mbO9NmzJyR)

#### エッジ検出

先程のエッジ検出のフィルタが何故下記のような値を用いるか考えます。  

$$
\begin{bmatrix} 
-1 & 0 & 1 \\\\ 
-1 & 0 & 1 \\\\
-1 & 0 & 1 
\end{bmatrix}
$$

まず物体のエッジ（輪郭）を捉えるためには、周囲の物体に対して人間がどのように判断しているかを考えてみましょう。おそらく、色合いが大きく異なる点で物体の輪郭を感じると思います。そのため、エッジを検出するためには、輝度の変化量を用いれば良いことが分かります。そして、変化量とは**微分**を使えば求めることができました。  

![エッジ検出](http://drive.google.com/uc?export=view&id=1ij6lWmBzt0CXIM4LNyp605SxWrffGewc)

上図のように、各座標 $x$ に対する輝度を表す関数 $f(x)$ があり、この勾配 $f^{'}(x)$ が変化量を表しています。ただし、画像内で座標情報から輝度情報を表現する $f(x)$ を求めることは現実的に困難です。一方、前後の座標における輝度 $f(x-1)$ と $f(x+1)$ は求まっているため、2 点を通る直線の傾きを $a$ とすると

$$
\begin{align}
a &= \frac{f(x+1) - f(x-1)}{(x+1) - (x-1)} \\\\
&=\frac{f(x+1) - f(x-1)}{2}
\end{align}
$$

となり、変化量を近似することができます。  

![25_22](http://drive.google.com/uc?export=view&id=1D-LULfIA278huIhVbaXWYq3xcNeVf4HW)

また、上図のように求まったフィルタを画像に対して掛けると、

$$
\begin{align}
&-1 × f(x-1) + 0 × f(x) +1 × f(x+1) \\\\
&= f(x+1) - f(x-1)
\end{align}
$$

が得られます。$a$ の定数項を無視すると、上の式より変化量として計算したい値と一致することがわかります。つまり、エッジ検出として紹介していたフィルタの値は適当に決めたものではなく、この変化量を取得したいという意図があって設定されたものであったと気づくことができました。  

エッジ検出ではこのような背景によって決定された値をフィルタに使用していることがわかりました。

## Convolutional Neural Network (CNN) の概要と構造

エッジ検出のフィルタの値はわかったのですが、犬と猫を判別するためのフィルタの値はいくらでしょうか。また、背景を除去することができるフィルタの値はいくらでしょうか。このように考えると、一見便利そうなフィルタですが、そのフィルタの値を決定することができなければ使うことができず、この**フィルタの値を決めることが難しいタスク**であるということがわかります。  

そのため、画像処理という学問が体系だっていましたがブレイクスルーを起こすことができませんでした。しかし、近年画像処理の領域でブレイクスルーが起きています。それが、畳み込みニューラルネットワーク (CNN) です。  

![従来のモデル](http://drive.google.com/uc?export=view&id=148PFGDBiNM6zMoxDy-Clt93xOqpEx3Uh)

従来の画像解析が上の流れだとすると、これから紹介する畳み込みニューラルネットワークでは、この畳み込みと全結合ニューラルネットワークの働きを一体化させ、**特徴抽出と分類器を学習するプロセスをすべてニューラルネットワークの中に包括**しました。これにより、これまで経験と勘によって行われていた前工程を自動化することができました。  

![CNN モデル](http://drive.google.com/uc?export=view&id=1qaAgtxdqL59tXFoff3eVgnPzuBIyOxQo)

畳み込みニューラルネットワークには、大きく以下の 3 つの処理を含んでいます。

- 畳み込み (Convolution)
- プーリング (Pooling)
- 全結合 (Fully Connected)

順を追って確認していきます。

### 畳み込み (Convolution)

畳み込みニューラルネットワークの核となるのが 畳み込み (Convolution) になります。これは前述したフィルタの計算を行う部分であり、入力情報の特徴を捉えます。エッジ検出ではフィルタの値がすべて決まっていましたが、畳み込みではすべて**学習すべきパラメータ**となっており、フィルタのサイズ、ストライドの値、パディングの有無がハイパーパラメータとなってます。また、畳み込み後に計算されたデータは**特徴マップ (Feature map)**と呼ばれます。

![25_24](http://drive.google.com/uc?export=view&id=13Sygt3wARh5jpBIQCh9O29IiXJycxOw-)

また、**フィルタ自体の数もハイパーパラメータになります。**フィルタの数を増やすことにより、特徴マップの数を増やす事ができ、取得できる情報量を増やす事が可能になります。    

### プーリング (Pooling)

畳み込まれた特徴マップを縮小させる処理を**プーリング (Pooling)** と呼びます。プーリング処理にはいくつか種類があり、

- MaxPooling
- AveragePooling
- GlobalAveragePooling

**MaxPooling** が最もよく使用されるので、最初はこちらを覚えておきましょう。処理内容を下図で表します。

![25_25](http://drive.google.com/uc?export=view&id=1ly1Q9GnUKlRMrS7dhjqyQigKe9XYDfC6)

あるサイズ（カーネルのサイズ）の中で最大の値か、平均の値か、どのような代表的な値を取るかで分かれています。前述した畳み込み処理とプーリング処理は続けて行うことが一般的であり、世界的な画像コンペティションでは複数回続けて処理することでモデルの性能を高めています。

### 全結合 (Fully Connected)

畳み込みとプーリングを複数回行った後、最終的に出力された複数のチャネルからなる画像の値を 1 列に並べ、ニューラルネットワークの入力層の値として用います。 全結合層の流れに関しては前章までで取り扱ったニューラルネットワークと同様になります。

### CNN モデルの計算の流れ

上記で確認したそれぞれの処理をまとめ、次章で実装を行うモデルの構造を確認します。  

![CNN 構造](http://drive.google.com/uc?export=view&id=1HuBwbjvsvSlORVVCV1voZoKiktPTzQnh)

処理それぞれの役割をもう一度整理して確認しましょう。  

- Convolution : 特徴量の抽出（前処理）
- Pooling : 特徴マップの圧縮
- Flatten : 特徴マップのベクトル化
- Dense : 分類器

Flatten は単純に特徴マップの値を一列に並べているだけになります。特徴量抽出の部分 (Convolution + Pooling) と分類を行う部分 (Dense) が 1 繋がりとなっているため、このように単純にならべるだけでもパラメータが調整され、分類を行うことができます。  

## 練習問題 本章のまとめ

本章で学んだ内容を復習しましょう。問題の答えをそれぞれのセルに記載してください。 （プログラミングを行うのではなく、問題をノート上などで解き、回答を空白のセルに記入してください。）

下記の CNN の処理の計算を行い、Flatten の処理が施された後のベクトルのサイズを計算して下さい。また、調整されるパラメータの数も算出して下さい。    
計算内容については下記の補足を確認して下さい。

![CNN 課題](http://drive.google.com/uc?export=view&id=1DXs523L0DoiU-gldas2e0kLiziE77KZa)

*補足*  

- 入力画像のチャンネル数は 1 チャンネルとします。
- 畳み込み層 (Convolution) のフィルタ数は 3 、ストライドは 1 、Padding は有りとします。
- Pooling 層のフィルタのサイズは $2 × 2$、ストライドは 2 とします。
- 今回パラメータ数の算出にはバイアスを考慮する必要はありません。

In [ ]:
# flatten 後のベクトルのサイズ


In [ ]:
# 調整するパラメータの総数


---
© 株式会社キカガク及び国立大学法人 豊橋技術科学大学